Summary Memory 기능을 통해서 AI와 나눈 대화 내용을 기억  
generation_llm의 결과를 받아서 메모리에 저장

## LLM 모델 정의

In [20]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model = "gpt-5-nano",
)

## 요약 기능이 포함되어 있는 노드 제작

In [21]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate

def summarize_old_messages(messages, max_messages=4):
    '''오래된 메세지를 LLM을 통해 요약하는 함수'''
    if len(messages) <= max_messages:   # 메세지가 적으면 반환시킴
        return messages
    
    old_messages = messages[:-max_messages]
    recent_messages = messages[-max_messages:]

    summary_prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 대화 내용을 간결하게 요약하는 전문가입니다.
            다음 대화 내용의 핵심 정보만 추출해서 3~5문장으로 요약해주세요.
            중요한 정보, 결정사항, 맥락을 유지화되 불필요한 세부사항은 생략하세요."""),
        ("user","다음 대화를 요약해주세요:\n\n{conversation}")
    ])

    conversation_text = ""
    for msg in old_messages:
        if msg.type == "human":
            conversation_text += f"사용자: {msg.content}\n"
        elif msg.type == "ai":
            conversation_text += f"AI: {msg.content}\n"
    
    try:
        summary_chain = summary_prompt | llm
        summary_response = summary_chain.invoke({"conversation": conversation_text})
        summary_content = summary_response.content
    except Exception as e:
        print(f"AI 요약 실패: {e}")
        summary_content = f"이전 대화 요약 ({len(old_messages)}개 메시지): \n{conversation_text[:500]}..."

    summary_message = SystemMessage(content=f"[이전 대화 요약]\n{summary_content}")

    return [summary_message] + recent_messages

In [22]:
from langgraph.graph import MessagesState

def chat_node_with_summary(state: MessagesState):
    summarized_messages = summarize_old_messages(state["messages"])

    response = llm.invoke(summarized_messages)
    return {"messages": [response]}

In [23]:
from langgraph.graph import StateGraph, MessagesState, START, END

graph_summary = StateGraph(MessagesState)
graph_summary.add_node("chat", chat_node_with_summary)
graph_summary.add_edge(START, "chat")
graph_summary.add_edge("chat", END)

In [24]:
from langgraph.checkpoint.memory import MemorySaver

memory_summary = MemorySaver()
app_with_summary = graph_summary.compile(checkpointer=memory_summary)

print("summary memory 기능 준비(오래된 대화는 요약해서 기억)")

summary memory 기능 준비(오래된 대화는 요약해서 기억)


In [25]:
from langchain_core.messages import HumanMessage
import uuid

memory_id = str(uuid.uuid4())
config_summary = {"configurable": {"thread_id": memory_id}}

config_summary

{'configurable': {'thread_id': '1602c752-1fe1-4d0c-a13f-f16c989579c2'}}

In [26]:
test_messages = [
    "내 이름은 김철수야",
    "나는 서울에 살아",
    "취미는 축구야",
    "오늘 날씨가 좋네",
    "좋아하는 과일은 수박, 복숭아야",
    "내 이름과 사는 곳이 뭐얐지?"
]

for i, msg in enumerate(test_messages, 1):
    print(f"{i}번째 대화: '{msg}'")
    result = app_with_summary.invoke(
        {"messages": [HumanMessage(content=msg)]},
        config=config_summary
    )
    print(f"AI: {result['messages'][-1].content}")
    print("-" * 40)

print("결과: 오래된 대화는 요약되지만 중요한 정보는 기억합니다!")
print("메모리 효율적이면서도 핵심 정보를 유지해요!")

1번째 대화: '내 이름은 김철수야'
AI: 반갑습니다, 김철수님. 무엇을 도와드릴까요? 글쓰기, 공부, 번역, 일정 관리 등 다양한 도움을 드릴 수 있어요. 예를 들어 이력서나 자기소개서 첨삭, 이메일 초안 작성, 자료 정리 등 구체적으로 어떤 도움이 필요하신지 말씀해 주시면 바로 도와드리겠습니다.
----------------------------------------
2번째 대화: '나는 서울에 살아'
AI: 좋아요! 서울에 계시다니 반갑습니다. 어떤 도움이 필요하신가요? 예를 들면:

- 이력서나 자기소개서 첨삭
- 이메일/메시지 초안 작성
- 공부 자료 정리, 요약, 노트 만들기
- 한국어 연습(대화, 문법, 어휘 안내)
- 일정 관리나 계획 세우기
- 서울 생활 팁(교통, 예산 관리, 맛집/카페 추천)

원하시는 주제나 상황을 알려주시면 바로 도와드릴게요. 또한 톤은 정중하게, 격식 있게, 또는 편하게 원하시는지 말씀해 주세요.
----------------------------------------
3번째 대화: '취미는 축구야'
AI: 멋져요! 서울에 살고 축구를 취미로 즐기신다니요.

원하는 도움 방향 몇 가지를 제안드려요. 원하시는 것을 골라 주시거나, 조합해도 좋습니다.

- 4주 축구 훈련 계획: 기술/피지컬/전술을 균형 있게 담은 주 3회 또는 주 4회 훈련안
- 경기 기록 노트 템플릿: 날짜/상대, 포지션, 득점/도움, 주의할 점 등 간단한 경기일지 양식
- 서울 내 축구 모임/클럽 찾기 방법: 동네 공원 축구, 유소년/성인 클럽, 코트 이용 팁
- 축구 관련 한국어 표현/용어 정리: 경기 중 쓰면 좋은 표현, 기사 읽기용 용어
- 축구 기사 요약 및 핵심 포인트 정리: 최신 이슈나 선수 인터뷰 간단 요약
- 실력 향상을 위한 개인 피드백 리스트: 현재 레벨(초보/중급/고급)과 목표에 맞춘 체크리스트

또 원하시면 바로 시작할 수 있도록 간단한 정보도 알려 주세요:
- 현재 실력대(초보/다져진 편/고급)
- 목표(예: 기술 향상, 체력